In [2]:
from main import load_and_preprocess_data

df_train, _ = load_and_preprocess_data("fraudTrain.csv")
df_test, _ = load_and_preprocess_data("fraudTest.csv")


Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%
Loading and preprocessing data...
Class distribution:
is_fraud
0    553574
1      2145
Name: count, dtype: int64
Fraud percentage: 0.39%


In [13]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from main import load_and_preprocess_data, SEQUENCE_LENGTH
from model import load_lstm_model
from predict import predict_and_analyze


fraud_users_train = df_train[df_train['is_fraud'] == 1]['cc_num'].unique()
fraud_users_test = df_test[df_test['is_fraud'] == 1]['cc_num'].unique()
print(len(fraud_users_train), len(fraud_users_test))

np.random.seed(42)
train_fraud_probs = []
train_non_fraud_probs = []
lstm_model, processor = load_lstm_model()
 
threshold = 0
threshold_train = []
for cc_num in fraud_users_train:
    user_transactions = df_train[df_train['cc_num'] == cc_num]

    user_non_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 0].iloc[:-1]
    user_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 1]

    if not user_fraud_transactions.empty:
        fraud_transaction = user_fraud_transactions.iloc[0].to_dict()
    else:
        print(f"No fraud transactions found for cc_num: {cc_num}")
        continue

    if not user_non_fraud_transactions.empty:
        non_fraud_transaction = user_non_fraud_transactions.iloc[-1].to_dict()
    else:
        print(f"No non-fraud transactions found for cc_num: {cc_num}")
        continue

    fraud_result = predict_and_analyze(
        lstm_model,
        user_non_fraud_transactions,
        SEQUENCE_LENGTH,
        processor,
        actual_transaction=fraud_transaction,
        fraud_transactions=user_fraud_transactions
    )

    non_fraud_result = predict_and_analyze(
        lstm_model,
        user_non_fraud_transactions,
        SEQUENCE_LENGTH,
        processor,
        actual_transaction=non_fraud_transaction,
        fraud_transactions=user_fraud_transactions
    )

    train_fraud_probs.append(fraud_result['total_probability'])
    train_non_fraud_probs.append(non_fraud_result['total_probability'])

    print(f"User: {cc_num}")
    print(f"Fraud Transaction Probability: {fraud_result['total_probability']:.10e}")
    print(f"Non-Fraud Transaction Probability: {non_fraud_result['total_probability']:.10e}")
    threshold_train.append(fraud_result['total_probability'])




762 218
Loading model from models/lstm_transaction_model.h5...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
User: 60416207185
Fraud Transaction Probability: 2.0929448142e-24
Non-Fraud Transaction Probability: 4.1557477459e-09
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
User: 60422928733
Fraud Transaction Probability: 3.2086711585e-22
Non-Fraud Transaction Probability: 2.2922548581e-10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
User: 60423098130
Fraud Transaction Probability: 1.6904945326e-13
Non-Fraud Transaction Probability: 1.8571371108e-09
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
User: 60427851591
Fraud Transaction Probability: 2.0327524155e-16
Non-Fraud Transaction Probability: 1.9093057801e-08
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
User: 60490596305
Fraud Transaction Probability: 1.2766161273e-22
Non-Fraud Transact

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


User: 213161869125933
Fraud Transaction Probability: 2.3328270328e-39
Non-Fraud Transaction Probability: 2.2029184560e-18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
User: 213163860545705
Fraud Transaction Probability: 1.4408273003e-37
Non-Fraud Transaction Probability: 6.3389706900e-25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
User: 213173753804333
Fraud Transaction Probability: 9.7813821654e-34
Non-Fraud Transaction Probability: 1.3458684906e-23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
User: 213174467670432
Fraud Transaction Probability: 5.2914078866e-30
Non-Fraud Transaction Probability: 5.2697067108e-25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
User: 213175487106448
Fraud Transaction Probability: 6.9173529951e-33
Non-Fraud Transaction Probability: 2.0108179080e-20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
User: 2131

...

User: 4742883543039288059
Fraud Transaction Probability: 4.2038953930e-45
Non-Fraud Transaction Probability: 8.4957222646e-39
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
User: 4756039869079882102
Fraud Transaction Probability: 3.1752723793e-26
Non-Fraud Transaction Probability: 2.8593604264e-13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
User: 4769426683924058915
Fraud Transaction Probability: 8.6497719251e-30
Non-Fraud Transaction Probability: 5.0011993109e-15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
User: 4792627764422477317
Fraud Transaction Probability: 7.7865968416e-21
Non-Fraud Transaction Probability: 3.5353427271e-14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
User: 4798937978667559470
Fraud Transaction Probability: 1.2329214373e-22
Non-Fraud Transaction Probability: 7.4781023388e-13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


User: 4906628655840914250
Fraud Transaction Probability: 8.8613910989e-41
Non-Fraud Transaction Probability: 6.4179561573e-30
No non-fraud transactions found for cc_num: 4911818930706644725
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
User: 4956828990005111019
Fraud Transaction Probability: 9.0661368029e-22
Non-Fraud Transaction Probability: 6.3920310017e-12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
User: 4973530368125489546
Fraud Transaction Probability: 2.9040424176e-37
Non-Fraud Transaction Probability: 1.6744758019e-19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
User: 4980323467523543940
Fraud Transaction Probability: 8.5453205037e-20
Non-Fraud Transaction Probability: 4.2757363439e-15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
User: 4992346398065154184
Fraud Transaction Probability: 1.2543253768e-38
Non-Fraud Transaction Probability: 1.5838956665e-12


In [ ]:
train_fraud_mean = np.mean(train_fraud_probs)
train_non_fraud_mean = np.mean(train_non_fraud_probs)
print(f"Train Fraud Mean Probability: {train_fraud_mean:.10e}")
print(f"Train Non-Fraud Mean Probability: {train_non_fraud_mean:.10e}")

threshold = 1e-24

Train Fraud Mean Probability: 1.1060699882e-12
Train Non-Fraud Mean Probability: 2.9813040925e-09


In [49]:
from sklearn.metrics import confusion_matrix, classification_report

y_true = []
y_pred = []

for cc_num in fraud_users_test:
    user_transactions = df_test[df_test['cc_num'] == cc_num]

    user_non_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 0].iloc[:-1]
    user_fraud_transactions = user_transactions[user_transactions['is_fraud'] == 1]

    if user_fraud_transactions.empty or user_non_fraud_transactions.empty:
        print(f"No transactions found for cc_num: {cc_num}")
        continue

    fraud_transaction = user_fraud_transactions.iloc[0].to_dict()
    non_fraud_transaction = user_non_fraud_transactions.iloc[-1].to_dict()

    fraud_prob = predict_and_analyze(
        lstm_model,
        user_non_fraud_transactions,
        SEQUENCE_LENGTH,
        processor,
        actual_transaction=fraud_transaction,
        fraud_transactions=user_fraud_transactions
    )['total_probability']

    non_fraud_prob = predict_and_analyze(
        lstm_model,
        user_non_fraud_transactions,
        SEQUENCE_LENGTH,
        processor,
        actual_transaction=non_fraud_transaction,
        fraud_transactions=user_fraud_transactions
    )['total_probability']

    # Actual: 1 = fraud, 0 = non-fraud
    # Predicted: 1 = predicted fraud (if legit probability is low)
    y_true.extend([1, 0])
    y_pred.extend([
        1 if fraud_prob <= threshold else 0,
        1 if non_fraud_prob <= threshold else 0
    ])
    print(f"cc_num: {cc_num}, Fraud Probability: {fraud_prob:.10e}, Non-Fraud Probability: {non_fraud_prob:.10e}")
    if fraud_prob <= threshold:
        print(f"Fraud detected for cc_num: {cc_num}")
    elif fraud_prob > threshold:
        print(f"No fraud detected for cc_num: {cc_num}")


print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, digits=4))





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
cc_num: 60487002085, Fraud Probability: 7.7748192114e-22, Non-Fraud Probability: 7.3175970491e-12
No fraud detected for cc_num: 60487002085
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
cc_num: 60495593109, Fraud Probability: 8.9077108591e-14, Non-Fraud Probability: 3.4295767615e-11
No fraud detected for cc_num: 60495593109
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
cc_num: 501882822387, Fraud Probability: 1.6098982017e-28, Non-Fraud Probability: 9.2934859808e-10
Fraud detected for cc_num: 501882822387
No transactions found for cc_num: 503851367360
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
cc_num: 570273151375, Fraud Probability: 5.8563902693e-22, Non-Fraud Probability: 1.9702280085e-10
No fraud detected for cc_num: 570273151375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
cc_num: 5718

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


cc_num: 30546453310167, Fraud Probability: 2.3969024268e-23, Non-Fraud Probability: 4.0233028226e-14
Fraud detected for cc_num: 30546453310167
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
cc_num: 30551643947183, Fraud Probability: 4.0091632262e-29, Non-Fraud Probability: 5.7997177011e-25
Fraud detected for cc_num: 30551643947183
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
cc_num: 30560609640617, Fraud Probability: 8.4724573245e-36, Non-Fraud Probability: 8.1314185456e-23
Fraud detected for cc_num: 30560609640617
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
cc_num: 30570197515105, Fraud Probability: 1.7007559462e-40, Non-Fraud Probability: 4.3238470530e-13
Fraud detected for cc_num: 30570197515105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
cc_num: 36913587729122, Fraud Probability: 4.5068712960e-30, Non-Fraud Probability: 7.6159629140e-15
Fraud detected for cc_n

...

cc_num: 3593118134380341, Fraud Probability: 3.2308193614e-34, Non-Fraud Probability: 5.1023049579e-20
Fraud detected for cc_num: 3593118134380341
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
cc_num: 3596217206093829, Fraud Probability: 3.7835058537e-44, Non-Fraud Probability: 3.3913871971e-20
Fraud detected for cc_num: 3596217206093829
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
cc_num: 3596357274378601, Fraud Probability: 5.7985730454e-42, Non-Fraud Probability: 1.3308559312e-22
Fraud detected for cc_num: 3596357274378601
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
cc_num: 3597926034019603, Fraud Probability: 0.0000000000e+00, Non-Fraud Probability: 1.0961633228e-32
Fraud detected for cc_num: 3597926034019603
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
cc_num: 4003989662068504, Fraud Probability: 9.3350065146e-28, Non-Fraud Probability: 9.6079274508e-24
Fraud

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_histograms_impl.py:895: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


cc_num: 4225990116481262579, Fraud Probability: 0.0000000000e+00, Non-Fraud Probability: 7.6265668921e-41
Fraud detected for cc_num: 4225990116481262579
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
cc_num: 4244953610136705533, Fraud Probability: 9.5579157649e-26, Non-Fraud Probability: 1.3521311115e-33
Fraud detected for cc_num: 4244953610136705533
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
cc_num: 4292743669224718067, Fraud Probability: 6.8195264450e-32, Non-Fraud Probability: 2.6090462146e-23
Fraud detected for cc_num: 4292743669224718067
No transactions found for cc_num: 4352307151555405069
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
cc_num: 4428780983793657331, Fraud Probability: 2.4695343366e-36, Non-Fraud Probability: 4.6070627971e-22
Fraud detected for cc_num: 4428780983793657331
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
cc_num: 4462038234743679432, F